In [15]:
import os 
import warnings
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns




In [17]:
from sklearn.metrics import mean_squared_error , r2_score
from sklearn.model_selection import train_test_split 

from sklearn.linear_model import LinearRegression

In [11]:
#  we are using the housing data 
#and we have already saved the training and testing data through previous excercise 

train_data_path = 'train.csv'
test_data_path = 'test.csv'

train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

In [21]:
train_data.head(2)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,rooms_per_household,bedrooms_per_room,population_per_household,ocean_proximity_INLAND,ocean_proximity_ISLAND,ocean_proximity_NEAR BAY,ocean_proximity_NEAR OCEAN
0,-121.46,38.52,29.0,3873.0,797.0,2237.0,706.0,2.1736,72100.0,5.485836,0.205784,3.168555,True,False,False,False
1,-117.23,33.09,7.0,5320.0,855.0,2015.0,768.0,6.3373,279600.0,6.927083,0.160714,2.623698,False,False,False,True


In [25]:
import mlflow 
import mlflow.sklearn

In [29]:
# Setting up the server URL for the MLflow server that is running on our system

remote_server_uri  = 'http://127.0.0.1:5000'

mlflow.set_tracking_uri(remote_server_uri)

In [31]:
#check weather it is set up or not 

mlflow.tracking.get_tracking_uri()

'http://127.0.0.1:5000'

In [33]:
# setting up the experiment 
exp_name  = "MLflow-walkthrough"
mlflow.set_experiment(exp_name)

2024/08/02 01:44:03 INFO mlflow.tracking.fluent: Experiment with name 'MLflow-walkthrough' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/550337396008449100', creation_time=1722543243228, experiment_id='550337396008449100', last_update_time=1722543243228, lifecycle_stage='active', name='MLflow-walkthrough', tags={}>